In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Big Data Sales Analysis") \
    .master("local[*]") \
    .getOrCreate()

spark


In [3]:
# Load Superstore sales data
df = spark.read.csv(
    "../data/superstore.csv",
    header=True,
    inferSchema=True
)

df.show(5)


+------+--------------+----------+----------+--------------+-----------+---------------+---------+-------------+---------------+----------+-----------+------+---------------+---------------+------------+--------------------+--------+--------+--------+--------+
|Row ID|      Order ID|Order Date| Ship Date|     Ship Mode|Customer ID|  Customer Name|  Segment|      Country|           City|     State|Postal Code|Region|     Product ID|       Category|Sub-Category|        Product Name|   Sales|Quantity|Discount|  Profit|
+------+--------------+----------+----------+--------------+-----------+---------------+---------+-------------+---------------+----------+-----------+------+---------------+---------------+------------+--------------------+--------+--------+--------+--------+
|     1|CA-2013-152156|09-11-2013|12-11-2013|  Second Class|   CG-12520|    Claire Gute| Consumer|United States|      Henderson|  Kentucky|      42420| South|FUR-BO-10001798|      Furniture|   Bookcases|Bush Somerset 

In [4]:
# Check schema (data types)
df.printSchema()

# Check column names
df.columns


root
 |-- Row ID: integer (nullable = true)
 |-- Order ID: string (nullable = true)
 |-- Order Date: string (nullable = true)
 |-- Ship Date: string (nullable = true)
 |-- Ship Mode: string (nullable = true)
 |-- Customer ID: string (nullable = true)
 |-- Customer Name: string (nullable = true)
 |-- Segment: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Postal Code: integer (nullable = true)
 |-- Region: string (nullable = true)
 |-- Product ID: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Sub-Category: string (nullable = true)
 |-- Product Name: string (nullable = true)
 |-- Sales: string (nullable = true)
 |-- Quantity: string (nullable = true)
 |-- Discount: string (nullable = true)
 |-- Profit: double (nullable = true)



['Row ID',
 'Order ID',
 'Order Date',
 'Ship Date',
 'Ship Mode',
 'Customer ID',
 'Customer Name',
 'Segment',
 'Country',
 'City',
 'State',
 'Postal Code',
 'Region',
 'Product ID',
 'Category',
 'Sub-Category',
 'Product Name',
 'Sales',
 'Quantity',
 'Discount',
 'Profit']

In [5]:
# Number of rows
df.count()

# Number of columns
len(df.columns)


21

In [6]:
from pyspark.sql.functions import col, sum

# Count nulls per column
null_counts = df.select([
    sum(col(c).isNull().cast("int")).alias(c) for c in df.columns
])

null_counts.show()


+------+--------+----------+---------+---------+-----------+-------------+-------+-------+----+-----+-----------+------+----------+--------+------------+------------+-----+--------+--------+------+
|Row ID|Order ID|Order Date|Ship Date|Ship Mode|Customer ID|Customer Name|Segment|Country|City|State|Postal Code|Region|Product ID|Category|Sub-Category|Product Name|Sales|Quantity|Discount|Profit|
+------+--------+----------+---------+---------+-----------+-------------+-------+-------+----+-----+-----------+------+----------+--------+------------+------------+-----+--------+--------+------+
|     0|       0|         0|        0|        0|          0|            0|      0|      0|   0|    0|          0|     0|         0|       0|           0|           0|    0|       0|       0|     0|
+------+--------+----------+---------+---------+-----------+-------------+-------+-------+----+-----+-----------+------+----------+--------+------------+------------+-----+--------+--------+------+



In [7]:
# Select key business columns
selected_cols = [
    "Order Date",
    "Region",
    "Category",
    "Sub-Category",
    "Sales",
    "Profit"
]

df_selected = df.select(selected_cols)
df_selected.show(5)


+----------+------+---------------+------------+--------+--------+
|Order Date|Region|       Category|Sub-Category|   Sales|  Profit|
+----------+------+---------------+------------+--------+--------+
|09-11-2013| South|      Furniture|   Bookcases|  261.96| 41.9136|
|09-11-2013| South|      Furniture|      Chairs|  731.94| 219.582|
|13-06-2013|  West|Office Supplies|      Labels|   14.62|  6.8714|
|11-10-2012| South|      Furniture|      Tables|957.5775|-383.031|
|11-10-2012| South|Office Supplies|     Storage|  22.368|  2.5164|
+----------+------+---------------+------------+--------+--------+
only showing top 5 rows



In [8]:
from pyspark.sql.functions import to_date

# Convert Order Date from string to date
df_selected = df_selected.withColumn(
    "Order Date",
    to_date("Order Date", "MM/dd/yyyy")
)

df_selected.printSchema()


root
 |-- Order Date: date (nullable = true)
 |-- Region: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Sub-Category: string (nullable = true)
 |-- Sales: string (nullable = true)
 |-- Profit: double (nullable = true)



In [9]:
# Check basic statistics for Sales and Profit
df_selected.select("Sales", "Profit").describe().show()


+-------+------------------+------------------+
|summary|             Sales|            Profit|
+-------+------------------+------------------+
|  count|              9994|              9994|
|   mean|234.41818199917006|28.587912967780834|
| stddev| 631.7890112674363| 234.3891156047269|
|    min|          10/Pack"|         -6599.978|
|    max|            999.98|          8399.976|
+-------+------------------+------------------+



In [10]:
# Remove rows where Sales or Profit is null (safety step)
df_clean = df_selected.dropna(subset=["Sales", "Profit"])

# Quick check
df_clean.count()


9994

In [11]:
from pyspark.sql.functions import sum

# Overall sales and profit
df_clean.select(
    sum("Sales").alias("Total_Sales"),
    sum("Profit").alias("Total_Profit")
).show()


+------------------+------------------+
|       Total_Sales|      Total_Profit|
+------------------+------------------+
|2272449.8562999545|285707.60220000165|
+------------------+------------------+



In [12]:
from pyspark.sql.functions import sum

# Region-wise sales and profit
df_clean.groupBy("Region").agg(
    sum("Sales").alias("Total_Sales"),
    sum("Profit").alias("Total_Profit")
).show()


+-------+------------------+------------------+
| Region|       Total_Sales|      Total_Profit|
+-------+------------------+------------------+
|  South|388983.58500000037|46650.341000000044|
|Central| 497800.8728000007| 40150.50299999999|
|   East| 672194.0539999981| 91603.05670000015|
|   West| 713471.3445000004|107303.70150000004|
+-------+------------------+------------------+



In [13]:
from pyspark.sql.functions import sum

# Category-wise sales and profit
df_clean.groupBy("Category").agg(
    sum("Sales").alias("Total_Sales"),
    sum("Profit").alias("Total_Profit")
).show()


+---------------+-----------------+------------------+
|       Category|      Total_Sales|      Total_Profit|
+---------------+-----------------+------------------+
|Office Supplies|703502.9280000031|120632.87839999991|
|      Furniture|733046.8612999996| 19686.42720000003|
|     Technology|835900.0669999964|145388.29659999989|
+---------------+-----------------+------------------+



In [14]:
from pyspark.sql.functions import sum

# Sub-Category-wise sales and profit
df_clean.groupBy("Sub-Category").agg(
    sum("Sales").alias("Total_Sales"),
    sum("Profit").alias("Total_Profit")
).show()


+------------+------------------+-------------------+
|Sub-Category|       Total_Sales|       Total_Profit|
+------------+------------------+-------------------+
|   Envelopes|15339.489999999993|  6461.269100000003|
|         Art|27118.791999999954|  6527.786999999998|
|      Chairs|328449.10300000076| 26590.166300000026|
| Furnishings| 82752.23000000004| 14294.297999999995|
|    Supplies| 45952.47000000001|-1347.3654999999983|
|   Fasteners|3008.6559999999995|  942.6377999999997|
|     Binders|199905.71700000006| 30038.821299999996|
|   Bookcases|114879.99629999997|-3472.5559999999978|
|      Labels|         12486.312|  5546.253999999998|
|       Paper| 75356.11799999999|  32795.56099999999|
| Accessories| 167380.3180000001|  41936.63569999993|
|     Copiers|149528.02999999994|  55617.82490000001|
|      Phones| 329753.0880000001|         44449.0791|
|    Machines|189238.63099999996|          3384.7569|
|     Storage|216803.21200000012|         21529.9083|
|  Appliances|        107532

In [15]:
from pyspark.sql.functions import sum

# Region + Category-wise sales and profit
df_clean.groupBy("Region", "Category").agg(
    sum("Sales").alias("Total_Sales"),
    sum("Profit").alias("Total_Profit")
).show()


+-------+---------------+------------------+------------------+
| Region|       Category|       Total_Sales|      Total_Profit|
+-------+---------------+------------------+------------------+
|  South|Office Supplies|123979.92499999993| 19595.75349999999|
|Central|Office Supplies|164616.19700000016| 9038.715399999997|
|  South|      Furniture| 116273.1360000001| 7071.571899999995|
|   West|Office Supplies|213125.18300000002| 51211.95060000014|
|Central|     Technology| 170401.5319999999|33693.441399999996|
|Central|      Furniture|162783.14380000005|        -2581.6538|
|  South|     Technology|148730.52399999992|19983.015600000006|
|   West|      Furniture|248450.23350000026|        11819.8689|
|   West|     Technology|251895.92799999993| 44271.88199999997|
|   East|Office Supplies|201781.62299999985| 40786.45889999996|
|   East|      Furniture|205540.34800000011|3376.6402000000016|
|   East|     Technology|264872.08300000033|47439.957599999936|
+-------+---------------+---------------

In [16]:
from pyspark.sql.functions import sum

# Top Region-Category pairs by Sales
df_clean.groupBy("Region", "Category").agg(
    sum("Sales").alias("Total_Sales"),
    sum("Profit").alias("Total_Profit")
).orderBy("Total_Sales", ascending=False).show(10)


+-------+---------------+------------------+------------------+
| Region|       Category|       Total_Sales|      Total_Profit|
+-------+---------------+------------------+------------------+
|   East|     Technology|264872.08300000033|47439.957599999936|
|   West|     Technology|251895.92799999993| 44271.88199999997|
|   West|      Furniture|248450.23350000026|        11819.8689|
|   West|Office Supplies|213125.18300000002| 51211.95060000014|
|   East|      Furniture|205540.34800000011|3376.6402000000016|
|   East|Office Supplies|201781.62299999985| 40786.45889999996|
|Central|     Technology| 170401.5319999999|33693.441399999996|
|Central|Office Supplies|164616.19700000016| 9038.715399999997|
|Central|      Furniture|162783.14380000005|        -2581.6538|
|  South|     Technology|148730.52399999992|19983.015600000006|
+-------+---------------+------------------+------------------+
only showing top 10 rows



In [17]:
from pyspark.sql.functions import sum

# Region-wise profit (focus on profitability)
df_clean.groupBy("Region").agg(
    sum("Profit").alias("Total_Profit")
).orderBy("Total_Profit", ascending=False).show()


+-------+------------------+
| Region|      Total_Profit|
+-------+------------------+
|   West|107303.70150000004|
|   East| 91603.05670000015|
|  South|46650.341000000044|
|Central| 40150.50299999999|
+-------+------------------+



In [18]:
from pyspark.sql.functions import sum

# Category-wise profit
df_clean.groupBy("Category").agg(
    sum("Profit").alias("Total_Profit")
).orderBy("Total_Profit", ascending=False).show()


+---------------+------------------+
|       Category|      Total_Profit|
+---------------+------------------+
|     Technology|145388.29659999989|
|Office Supplies|120632.87839999991|
|      Furniture| 19686.42720000003|
+---------------+------------------+



In [19]:
from pyspark.sql.functions import sum

# Region + Category profitability
df_clean.groupBy("Region", "Category").agg(
    sum("Profit").alias("Total_Profit")
).orderBy("Total_Profit", ascending=False).show()


+-------+---------------+------------------+
| Region|       Category|      Total_Profit|
+-------+---------------+------------------+
|   West|Office Supplies| 51211.95060000014|
|   East|     Technology|47439.957599999936|
|   West|     Technology| 44271.88199999997|
|   East|Office Supplies| 40786.45889999996|
|Central|     Technology|33693.441399999996|
|  South|     Technology|19983.015600000006|
|  South|Office Supplies| 19595.75349999999|
|   West|      Furniture|        11819.8689|
|Central|Office Supplies| 9038.715399999997|
|  South|      Furniture| 7071.571899999995|
|   East|      Furniture|3376.6402000000016|
|Central|      Furniture|        -2581.6538|
+-------+---------------+------------------+



## Business Insight
Technology category drives the highest profit, especially in East and West regions.
